# Floating Systematics

In this example we start with a signal and background with assumed probability density 
functions.
    $$ P(x;\mu, \sigma) = \exp\left(-\frac{(x-\mu)^2}
    {2\sigma^2}\right)/\sqrt{2\pi\sigma^2}$$

We will fit the fake dataset using an extended likelihood with the above PDFs.

### Analytic Fit
Float $\mu$, $\sigma$, and $N$ for both background and signal and fit using the
extended likelihood objective.

### Systematics
Include a form for the systematic and float the systematic, while keeping $\mu$ and 
$\sigma$ fixed.

### Numeric PDF
Use a DataFrame as the PDF instead of the analytic form, then perform the fits

In [ ]:
include("./src/WatchFish.jl")
using .WatchFish

In [ ]:
## Define an analytic spectrum to pass through
function spectrum(x, μ, σ)
    exp(-(x-μ)^2/2/σ^2)/sqrt(2*π*σ)
end

m = Model()

add_component!(m, "Signal", 20.0; spectrum=spectrum)
add_component!(m, "Background", 30.0; spectrum=spectrum)

In [ ]:
# Pseudo Code

# Begin a new model called m
m = Model()

# We have a data set already in a dataframe,
# we want to fit this dataset to a probability distribution.
@observable(m, energy)
# There are a number of parameters
@parameter(m, mean)
@parameter(m, resolution)
@parameter(m, number)

# The pdf of these parameters is
function p1(energy)
    exp(-(energy-mean)^2/2/resolution^2)/sqrt(2*π*resolution^2)
end

# I also have another component, call it background
@parameter(m, bgmean)
@parameter(m, bgres)
@parameter(m, bgnum)
function p2(energy)
    exp(-(energy-bgmean)^2/2/bgres^2)/sqrt(2*π*bgres^2)
end

d1 = DataFrame(energy=randn)
dataset!(m, d1)

function objective( data )
    x = data.energy
    λ = number + bgnum
    N = len(x)
    λ - sum( log.( number*p1(x) + bgnum*p2(x) ) ) + N*log(N) - N
end

@objective(m, objective)
optimize!(m)

## How would this same thing look for the poisson case?

p = Model()
@observable(p, count)
@parameter(p, s) # Signal
@parameter(p, b) # Background
dataset!(p, DataFrame(count=200))
function objective(data)
    k = data
    λ = s + b
    λ - N*log(λ) + N*log(N) - N
end
@objective(p, objective)
optimize!(p)

## What about constraints?
@constant(p, μ)
@constant(p, σ)
function sideband()
    (x-μ)^2/2/σ^2
end
@logconstraint(p, sideband)
# Should I explicitly add this?

## Convoluted Example

1. Multiple dimensions in observable $\vec{x_{1,2}}$
2. Multiple datasets $d_{1,2}$ of events $n_{1,2}$
3. Parameters $s$, $b$, $\mu$, $\sigma$

### Calibration
\begin{align}
\mathcal{O} = &(s+b) - n_1\log(sS(\vec{x_1})+bB(\vec{x_1})) 
+ \frac{(b-\mu)^2}{2\sigma^2} \\
 &+ \frac{(\vec{x_2}-\mu)^2}{2\sigma^2}
\end{align}

In [ ]:
using DataFrames
using Random

d = randn(100)
df = DataFrame(
    energy = d
)

function run_likelihood!(m::Model, data::DataFrame)
end

In [ ]:
# Testing: I would like to store observables themselves as
# some kind of object (struct)
using BenchmarkTools
using DataFrames
using NLopt

struct observable
    name::String
end

function f(x, μ, σ)
    exp(-(x-μ)^2/2/σ^2)/sqrt(2*π*σ^2)
end


x = observable("x")
y = observable("y")

# Using this I can construct an objective function with same 
# name data

r1 = randn(100)
#df = DataFrame(x=r1, y=r2)

function objective(x::Vector, grad::Vector)
    if length(grad)>0
        grad = 2x
    end
    sum( -log.(f.(r1, x[1], x[2])))
end

opt = Opt(:LN_SBPLX, 2)
opt.ftol_rel = 1e-4
opt.min_objective = objective
@benchmark optimize!(opt, [1.0, 1.0])
# NLopt wants an object of this form:
# function objective(x::Vector, grad::Vector)

In [ ]:
# Testing: I would like to store observables themselves as
# some kind of object (struct)
using BenchmarkTools
using DataFrames
using NLopt

struct observable
    name::String
end

function f(x, μ, σ)
    exp(-(x-μ)^2/2/σ^2)/sqrt(2*π*σ^2)
end


x = observable("x")
y = observable("y")

# Using this I can construct an objective function with same 
# name data

r1 = randn(100)
df = DataFrame(x=r1)

## Lets say we know x::observable
function g(μ, σ)
    exp(-(df.x-μ)^2/2/σ^2)/sqrt(2*π*σ^2)
end

function objective(x::Vector, grad::Vector)
    if length(grad)>0
        grad = 2x
    end
    sum( -log.(g.(x[1], x[2])))
end

opt = Opt(:LN_SBPLX, 2)
opt.ftol_rel = 1e-4
opt.min_objective = objective
@benchmark optimize!(opt, [1.0, 1.0])
# NLopt wants an object of this form:
# function objective(x::Vector, grad::Vector)